In [1]:
import ee
import geemap
import random

In [2]:
L7bands = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7']

In [3]:
Map = geemap.Map()
# Map

In [4]:
def maskL7sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    # Return the masked image, scaled to TOA reflectance, without the QA bands.
    return image.updateMask(mask).divide(10000).select(L8bands).copyProperties(image, ["system:time_start"])

In [5]:
# Creating an add variable function for Landsat 7 index calculation.
def compute_indices(image):
    NDVI = image.expression('(B4-B3)/(B4+B3)', 
                            {'B3': image.select('B3'),
                             'B4': image.select('B4')}).rename('ndvi')
    MNDWI = image.expression('(B2-B5) / (B2+B5)',
                             {'B2': image.select('B2'),
                              'B5': image.select('B5')}).rename('mndwi')
    EVI = image.expression('2.5 * ((B4 - B3)/(B4 +(6*B3)-(7.5*B1)+1))',
                           {'B1': image.select('B1'),
                            'B3': image.select('B3'),
                            'B4': image.select('B4')}).rename('evi')
    LSWI = image.expression('(B4-B5)/(B4+B5)', 
                            {'B4': image.select('B4'),
                             'B5': image.select('B5')}).rename('lswi')
    return image.addBands(NDVI).addBands(MNDWI).addBands(EVI).addBands(LSWI)

In [6]:
point = ee.Geometry.Point([106.4943, 20.4858])

image = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .filterBounds(point) \
    .filterDate('2010-01-01', '2010-12-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B3', 'B2', 'B1']
}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-7")
# Map

In [7]:
# Add the boundary of Thai Binh province
# red_river_delta_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Provinces_Polygon/Provinces_Poly.shp'
red_river_delta_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Provinces_PolygonN/Provinces_Poly.shp'
red_river_delta = geemap.shp_to_ee(red_river_delta_shp)

In [8]:
Map.addLayer(red_river_delta, {},"TB+ND+HP provinces")
Map.centerObject(red_river_delta, 9)
Map

Map(center=[20.50075230416886, 106.47941624308001], controls=(WidgetControl(options=['position'], widget=HBox(…

In [9]:
image_clip = image.clip(red_river_delta)
# Map.centerObject(point, 8)
# Map.addLayer(image_clip, vis_params, "Landsat-8")
# Map

In [10]:
img_compute = compute_indices(image_clip)

In [11]:
image_select = img_compute.select('ndvi', 'mndwi', 'evi', 'lswi')

In [12]:
# Add training markers from shape files
# agriculture_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Markers 20201205/Training/Agriculture_Training.shp'
# evergreen_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Markers 20201205/Training/Evergreen_Training.shp'
# mangrove_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Markers 20201205/Training/Mangrove.shp'
# others_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Markers 20201205/Training/Other.shp'
# residence_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Markers 20201205/Training/Residental area_Training.shp'
# shrimp_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Markers 20201205/Training/Shrimp.shp'
# water_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Markers 20201205/Training/Water_Training.shp'

agriculture_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Sample_final/Sample/Training/Agriculture_Training.shp'
evergreen_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Sample_final/Sample/Training/Evergreen_Training.shp'
mangrove_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Sample_final/Sample/Training/Mangrove.shp'
others_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Sample_final/Sample/Training/Other.shp'
residence_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Sample_final/Sample/Training/Residental area_Training.shp'
shrimp_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Sample_final/Sample/Training/Shrimp.shp'
water_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Sample_final/Sample/Training/Water_Training.shp'

# D:\Software\Jupyter\Thesis\Learning\GEE\Thuy\Sample_final\Sample\Training

# agriculture_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/agriculture_markers.shp'
# evergreen_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/evergreen_markers.shp'
# mangrove_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/mangrove_markers.shp'
# others_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/other_markers.shp'
# residence_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/residence_markers.shp'
# shrimp_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/shrimp_markers.shp'
# water_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/water_markers.shp'
    
# Shape files to ee objects
tr_agriculture = geemap.shp_to_ee(agriculture_shp)
tr_evergreen = geemap.shp_to_ee(evergreen_shp)
tr_mangrove = geemap.shp_to_ee(mangrove_shp)
tr_others = geemap.shp_to_ee(others_shp)
tr_residence = geemap.shp_to_ee(residence_shp)
tr_shrimp = geemap.shp_to_ee(shrimp_shp)
tr_water = geemap.shp_to_ee(water_shp)

# Make markers collections
tr_classNames = tr_agriculture.merge(tr_evergreen).merge(tr_mangrove). \
                               merge(tr_others).merge(tr_residence). \
                               merge(tr_shrimp).merge(tr_water)
# Show the markers on map
Map.addLayer(tr_classNames, {}, 'Markers')
Map

Map(center=[20.50075230416886, 106.47941624308001], controls=(WidgetControl(options=['position'], widget=HBox(…

In [13]:
# Set bands and label for classifier
bands = ['ndvi', 'mndwi', 'evi', 'lswi']
label = 'landcover'

In [14]:
sample = image_select.select(bands).sampleRegions(**{
    'collection': tr_classNames,
    'properties': [label],
    'scale': 30
})

In [15]:
# Adds a column of deterministic pseudorandom numbers. 
random.seed(2020)
sample = sample.randomColumn()
split = 0.7

In [16]:
# get training data which is 0.7 of sample
training = sample.filter(ee.Filter.lt('random', split))

In [17]:
# get validation/test data which is 0.3 of sample
validation = sample.filter(ee.Filter.gte('random', split))

In [18]:
# classify training data by randomForest
# img_classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)
# classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)
classifier = ee.Classifier.libsvm(kernelType= 'RBF', gamma= 0.5, cost= 10).train(training, label, bands)

In [19]:
# Run the classification
classified = image_select.select(bands).classify(classifier)

In [20]:
# Display classification
cl_vis_params = {
    'min': 1, 
    'max': 7, 
    'palette': ['#98ff00','#0b4a8b','#ffc82d','#00ffff','#bf04c2','#ff0000','#008800']
}

legend_keys = ['Mangrove', 'Shrimp', 'Residence', 'Water','Agriculture','Other','Evergreen']
legend_colors = ['#98ff00','#0b4a8b','#ffc82d','#00ffff','#bf04c2','#ff0000','#008800']

Map.centerObject(red_river_delta, 10)
Map.addLayer(classified, cl_vis_params, 'classification')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')
Map

Map(center=[20.50075230416886, 106.47941624308001], controls=(WidgetControl(options=['position'], widget=HBox(…

In [21]:
# Training dataset
train_accuracy = classifier.confusionMatrix()

In [22]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 401, 72, 5, 1, 47, 0, 1],
 [0, 63, 146, 24, 13, 15, 0, 0],
 [0, 13, 3, 130, 0, 7, 0, 0],
 [0, 32, 21, 1, 32, 2, 0, 0],
 [0, 81, 4, 35, 0, 175, 0, 2],
 [0, 19, 1, 40, 0, 9, 0, 0],
 [0, 6, 0, 0, 0, 0, 0, 87]]

In [23]:
train_accuracy.accuracy().getInfo()

0.6525537634408602

In [24]:
train_accuracy.kappa().getInfo()

0.5478623115152599

In [25]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [0.7609108159392789],
 [0.5593869731800766],
 [0.8496732026143791],
 [0.36363636363636365],
 [0.5892255892255892],
 [0],
 [0.9354838709677419]]

In [26]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  0.6520325203252032,
  0.5910931174089069,
  0.5531914893617021,
  0.6956521739130435,
  0.6862745098039216,
  0,
  0.9666666666666667]]

In [27]:
# Validation dataset
validated = validation.classify(classifier)

In [28]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '1_1_1_1_1_1_0_0',
 'properties': {'classification': 1,
  'evi': 0.22936698717948717,
  'landcover': 5,
  'lswi': 0.6184296011924744,
  'mndwi': 0.4991482198238373,
  'ndvi': 0.1011037528514862,
  'random': 0.9051248993813626}}

In [29]:
test_accuracy = validated.errorMatrix('landcover', 'classification')

In [30]:
test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 164, 21, 2, 1, 26, 0, 4],
 [0, 26, 50, 5, 7, 6, 0, 0],
 [0, 9, 3, 50, 0, 0, 0, 1],
 [0, 13, 9, 0, 10, 4, 0, 0],
 [0, 44, 6, 13, 0, 80, 0, 1],
 [0, 6, 3, 15, 0, 3, 0, 0],
 [0, 5, 1, 0, 0, 1, 0, 28]]

In [31]:
test_accuracy.accuracy().getInfo()

0.6191247974068071

In [32]:
test_accuracy.kappa().getInfo()

0.49875202234605975

In [33]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [0.7522935779816514],
 [0.5319148936170213],
 [0.7936507936507936],
 [0.2777777777777778],
 [0.5555555555555556],
 [0],
 [0.8]]

In [34]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  0.6142322097378277,
  0.5376344086021505,
  0.5882352941176471,
  0.5555555555555556,
  0.6666666666666666,
  0,
  0.8235294117647058]]

In [35]:
import csv
import os

out_dir = os.path.join(os.path.expanduser('D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/'), 'Results')
#D:\Software\Jupyter\Thesis\Learning\GEE\Thuy
training_csv = os.path.join(out_dir, 'train_accuracy_rf.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy_rf.csv')

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())
    
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())

In [36]:
# Export the result

In [37]:
import os
out_dir = os.path.join(os.path.expanduser('D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/'), 'Results')
#D:\Software\Jupyter\Thesis\Learning\GEE\Thuy
out_file = os.path.join(out_dir, 'landcover_rf.tif')

In [38]:
# Export the result directly to your computer:
geemap.ee_export_image(classified, filename=out_file, scale=900)

Generating URL ...
Please wait ...
Data downloaded to D:\Software\Jupyter\Thesis\Learning\GEE\Thuy\Results\landcover_rf.tif
